## Taller 2

### Import modules

In [1]:
# Import modules from python standard library
import os
import pprint
import random

# Import Geopandas for vector analysis (https://geopandas.org)
import geopandas

# Import the matplotlib plotting module and set it to work within
# a notebook environment.
%matplotlib inline
import matplotlib
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Import the RSGISLib modules for the notebook
import rsgislib
import rsgislib.classification
import rsgislib.classification.classsklearn
import rsgislib.tools.filetools
import rsgislib.tools.mapping
import rsgislib.tools.plotting
import rsgislib.vectorutils
import rsgislib.zonalstats
from sklearn.ensemble import RandomForestClassifier

# scikit-learn imports
from sklearn.model_selection import GridSearchCV

Cargar imagen Sentinel-2

In [ ]:
# Input Sentinel-2 image file with 20 m resolution pixels.
input_img = "notebooks/Data_geoprocessing/data/sen2_20180629_t30uvd_orb037_osgb_stdsref_20m.tif"

# A valid pixel mask for the input image - this is required for later
# steps but note that for this particular image all the pixel values
# are valid (i.e., there aren't are no data regions).
in_msk_img = "notebooks/Data_geoprocessing/sen2_20180629_t30uvd_orb037_osgb_stdsref_20m_vmsk.tif"

vec_train_file = "notebooks/Data_geoprocessing/aber_sen2_cls_training.gpkg" #Datos verdaderos